In [12]:
import torch
import networkx as nx
import numpy as np
import scipy.sparse as sp
import pandas as pd
from math import ceil
import glob
import unidecode 
from datetime import date, timedelta

from sklearn import preprocessing

import os

def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if (max_value - min_value) != 0:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        else:
            result[feature_name] = df[feature_name]
            
    return result

os.chdir("../data")
window=7
scaled=False

Gs = []
d = pd.read_excel("SC_Graph_synthetic.xlsx", sheet_name="Edge")
del d['Time']
nd = normalize(d)

for Graphs in nd.iterrows():
    G = nx.DiGraph()
    for i in range(0,len(Graphs[1]),3):
        G.add_edge(Graphs[1][i], Graphs[1][i+1], weight=Graphs[1][i+2])
    Gs.append(G)

suppliersnn = pd.read_excel("SC_Graph_synthetic.xlsx", sheet_name="Suppliers")
manufacturersnn = pd.read_excel("SC_Graph_synthetic.xlsx", sheet_name="Manufacturers")
distributorsnn = pd.read_excel("SC_Graph_synthetic.xlsx", sheet_name="Distributors")
customersnn = pd.read_excel("SC_Graph_synthetic.xlsx", sheet_name="Customers")
suppliers = normalize(suppliersnn)
manufacturers = normalize(manufacturersnn)
distributors = normalize(distributorsnn)
customers = normalize(customersnn)

features = list()

nodes = Gs[0].nodes()
  
#--- one hot encoded the region
#departments_name_to_id = dict()
#for node in nodes:
#departments_name_to_id[node] = len(departments_name_to_id)
#n_departments = len(departments_name_to_id)
    
#print(n_departments)
for idx,G in enumerate(Gs):
#  Features = population, coordinates, d past cases, one hot region
        
    H = np.zeros([G.number_of_nodes(),window]) #+3+n_departments])#])#])
    
    ### enumarate because H[i] and labs[node] are not aligned
    #---- Past cases      
    if(idx < window):# idx-1 goes before the start of the labels
        H[0,(window-idx):(window)] = suppliers['Supplier_001_Capacity'][0:(idx)]
        H[1,(window-idx):(window)] = suppliers['Supplier_002_Capacity'][0:(idx)]
        H[2,(window-idx):(window)] = suppliers['Supplier_003_Capacity'][0:(idx)]
        H[3,(window-idx):(window)] = suppliers['Supplier_004_Capacity'][0:(idx)]
        H[4,(window-idx):(window)] = suppliers['Supplier_005_Capacity'][0:(idx)]
        H[5,(window-idx):(window)] = manufacturers['Manufacturer_001_Capacity'][0:(idx)]
        H[6,(window-idx):(window)] = manufacturers['Manufacturer_002_Capacity'][0:(idx)]
        H[7,(window-idx):(window)] = manufacturers['Manufacturer_003_Capacity'][0:(idx)]
        H[8,(window-idx):(window)] = manufacturers['Manufacturer_004_Capacity'][0:(idx)]
        H[9,(window-idx):(window)] = manufacturers['Manufacturer_005_Capacity'][0:(idx)]
        H[10,(window-idx):(window)] = distributors['Distributor_001_Inventory'][0:(idx)]
        H[11,(window-idx):(window)] = distributors['Distributor_002_Inventory'][0:(idx)]
        H[12,(window-idx):(window)] = distributors['Distributor_003_Inventory'][0:(idx)]
        H[13,(window-idx):(window)] = distributors['Distributor_004_Inventory'][0:(idx)]
        H[14,(window-idx):(window)] = distributors['Distributor_005_Inventory'][0:(idx)]
        H[15,(window-idx):(window)] = distributors['Distributor_006_Inventory'][0:(idx)]
        H[16,(window-idx):(window)] = customers['Customer_001_Inventory'][0:(idx)]
        H[17,(window-idx):(window)] = customers['Customer_002_Inventory'][0:(idx)]
        H[18,(window-idx):(window)] = customers['Customer_003_Inventory'][0:(idx)]
        H[19,(window-idx):(window)] = customers['Customer_004_Inventory'][0:(idx)]
                
    elif idx >= window:
        H[0,0:(window)] = suppliers['Supplier_001_Capacity'][(idx-window):(idx)]
        H[1,0:(window)] = suppliers['Supplier_002_Capacity'][(idx-window):(idx)]
        H[2,0:(window)] = suppliers['Supplier_003_Capacity'][(idx-window):(idx)]
        H[3,0:(window)] = suppliers['Supplier_004_Capacity'][(idx-window):(idx)]
        H[4,0:(window)] = suppliers['Supplier_005_Capacity'][(idx-window):(idx)]
        H[5,0:(window)] = manufacturers['Manufacturer_001_Capacity'][(idx-window):(idx)]
        H[6,0:(window)] = manufacturers['Manufacturer_002_Capacity'][(idx-window):(idx)]
        H[7,0:(window)] = manufacturers['Manufacturer_003_Capacity'][(idx-window):(idx)]
        H[8,0:(window)] = manufacturers['Manufacturer_004_Capacity'][(idx-window):(idx)]
        H[9,0:(window)] = manufacturers['Manufacturer_005_Capacity'][(idx-window):(idx)]
        H[10,0:(window)] = distributors['Distributor_001_Inventory'][(idx-window):(idx)]
        H[11,0:(window)] = distributors['Distributor_002_Inventory'][(idx-window):(idx)]
        H[12,0:(window)] = distributors['Distributor_003_Inventory'][(idx-window):(idx)]
        H[13,0:(window)] = distributors['Distributor_004_Inventory'][(idx-window):(idx)]
        H[14,0:(window)] = distributors['Distributor_005_Inventory'][(idx-window):(idx)]
        H[15,0:(window)] = distributors['Distributor_006_Inventory'][(idx-window):(idx)]
        H[16,0:(window)] = customers['Customer_001_Inventory'][(idx-window):(idx)]
        H[17,0:(window)] = customers['Customer_002_Inventory'][(idx-window):(idx)]
        H[18,0:(window)] = customers['Customer_003_Inventory'][(idx-window):(idx)]
        H[19,0:(window)] = customers['Customer_004_Inventory'][(idx-window):(idx)]
        
    features.append(H)
print(len(features))
print(features)

360
[array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]]), array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.56857353],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.50507945],
       [0.        , 0.        , 0.        , 0.     